## Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import easyocr
from PIL import Image
import glob
import promptlib
from time import sleep
from tqdm.auto import tqdm
from progress.bar import Bar
import glob
import re
from collections import OrderedDict
import string
import csv
import pandas as pd
reader=easyocr.Reader(['en'])
import time


### Framing Function

In [18]:
def getFrame(sec,file_name,count):
    vidcap = cv2.VideoCapture(file_name)
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(r"E:\MS 4th Semester\Thesis\1st Meeting (8th Jan)\Syed Irtza Akhtar Bukhari - OCR patient monitor\irtza\frames"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames

### Pre Processing (HSV, Color Masking, Finding Countours)

In [19]:
def hsv_conversion(image):
    hsv_frame=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    return hsv_frame

In [20]:
def get_green_mask(hsv_frame,image):
    low_green=np.array([50, 25, 25])       #low green threshold value for green color masking  (Can manually adjust)
    high_green=np.array([70, 255,255])     #High green threshold value for green color masking (Can manually adjust)
    mask_green=cv2.inRange(hsv_frame,low_green,high_green)
    green=cv2.bitwise_and(image,image,mask=mask_green)
    return green

In [21]:
def get_yellow_mask(hsv_frame,image):
    low_yellow=np.array([25, 80, 80])     #low yellow threshold value for yellow color masking  (Can manually adjust)
    high_yellow=np.array([40, 255, 255])  #High yellow threshold value for yellow color masking  (Can manually adjust)
    mask_yellow=cv2.inRange(hsv_frame,low_yellow,high_yellow)
    yellow=cv2.bitwise_and(image,image,mask=mask_yellow)
    return yellow

In [22]:
def get_red_mask(hsv_frame,image):
    low_red1=np.array([0, 70, 50])        #Red threshold value for red color masking  (Can manually adjust)
    high_red1=np.array([10, 255, 255])
    low_red2=np.array([170, 70, 50])
    high_red2=np.array([180, 255, 255])

    mask_red1=cv2.inRange(hsv_frame,low_red1,high_red1)
    mask_red2=cv2.inRange(hsv_frame,low_red2,high_red2)
    mask_red=mask_red1|mask_red2
    red=cv2.bitwise_and(image,image,mask=mask_red)
    return red

In [72]:
def get_white_mask(hsv_frame,image):
    low_white=np.array([0,0,168])      #White color masking
    high_white=np.array([172,111,255])
    mask_white=cv2.inRange(hsv_frame,low_white,high_white)
    white=cv2.bitwise_and(image,image,mask=mask_white)
    return white

In [24]:
def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

### OCR Implementation (Detecting Headers)
#### Functions to detect only the alphanumeric characters which we could save as the main header of the key paramter in a dataframe

In [25]:
def get_mix_strings(item):
    for ch in item:
        if not ch.isalpha():
            if not ch.isdigit():
                return False
    return True

def get_all_num(item):
    return all([ch.isdigit() for ch in item])

def get_all_alpha(item):
    return all([ch.isalpha() for ch in item])

In [106]:
def OCR_header(color):
    result = reader.readtext(color,min_size= 25,detail=0,text_threshold=0.90)  #Apply easyOCR to detect all the characters
    final_header = []
    for item in result:
        if get_all_num(item):
            continue
        elif get_mix_strings(item):
            final_header.append(item)
        elif get_all_alpha(item):
            final_header.append((item))
            
    return final_header

### OCR Implementation (Detecting Values against headers)

In [121]:
def OCR_values(color):
    result = reader.readtext(color,min_size= 80,detail=0,text_threshold=0.90)
    value = [x for x in result if all(x1.isdigit() for x1 in x)]   #Select only digit values to store in a corresponding key
    return value

### Header Dataframe Function

In [46]:
def get_headers_dataframe(header_color):
    final_header=['HR','Tperi','Tblood','Pulse','SpO2','ABP','PAP','etCO2','awRR','NBP','etCO2','mCO2']
    header=[]
    for i in header_color:
        for j in final_header:
            if i==j:
                header.append(j)
    
    header=list(OrderedDict.fromkeys(header))
    df = pd.DataFrame(columns = header)
    return df,header

### Numeric Values in dataframe

In [130]:
def df_values(df,header,value):
    
    if len(value)<len(header):
        value.extend([str(0)]*(len(header)-len(value)))
    length_df = len(df)
    df.loc[length_df] = value[:len(header)]
    return df

### Main function to call Preprocessing and OCR 

In [125]:
def main():
    prompter = promptlib.Files()
    file_name = prompter.file()
    sec = 0
    frameRate = 1 #//it will capture one image in each 1 second
    count=1
    success = getFrame(sec,file_name,count)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec,file_name,count)
    # Color masking and OCR implementation to create dataframe and make headers (One time process only)   
    print("Framing done")
    image_header=cv2.imread(r'E:\MS 4th Semester\Thesis\1st Meeting (8th Jan)\Syed Irtza Akhtar Bukhari - OCR patient monitor\irtza\frames200.jpg')
    hsv_frame=hsv_conversion(image_header)
    green=get_green_mask(hsv_frame,image_header)
    red=get_red_mask(hsv_frame,image_header)
    yellow=get_yellow_mask(hsv_frame,image_header)
    white=get_white_mask(hsv_frame,image_header)
    header_green=OCR_header(color=green)
    header_red=OCR_header(color=red)
    header_yellow=OCR_header(color=yellow)
    header_white=OCR_header(color=white)
    
    df_red,hd_red=get_headers_dataframe(header_red)
    df_yellow,hd_yellow=get_headers_dataframe(header_yellow)
    df_white,hd_white=get_headers_dataframe(header_white)
    df_green,hd_green=get_headers_dataframe(header_green)
    
    # Save all numeric values of corresponding headers detected earlier to update dataframe 
    for infile in sorted(glob.glob(r'E:\MS 4th Semester\Thesis\1st Meeting (8th Jan)\Syed Irtza Akhtar Bukhari - OCR patient monitor\irtza\*.jpg'), key=numericalSort):
        image = cv2.imread(infile)

        hsv_frame=hsv_conversion(image)
        green=get_green_mask(hsv_frame,image)
        red=get_red_mask(hsv_frame,image)
        yellow=get_yellow_mask(hsv_frame,image)
        white=get_white_mask(hsv_frame,image)
        
        green_value=OCR_values(green)
        red_value=OCR_values(red)
        yellow_value=OCR_values(yellow)
        white_value=OCR_values(white)
        
        df_green=df_values(df_green,hd_green,green_value)
        df_red=df_values(df_red,hd_red,red_value)
        df_yellow=df_values(df_yellow,hd_yellow,yellow_value)
        df_white=df_values(df_white,hd_white,white_value)
    # Save the dataframe into .csv file.
    df=pd.concat([df_green,df_yellow,df_red,df_white],axis=1)
    df.to_csv(r'E:\MS 4th Semester\Thesis\1st Meeting (8th Jan)\Syed Irtza Akhtar Bukhari - OCR patient monitor\irtza\Output.csv', index = False)
    
    return df

In [ ]:
start = time.time()
df=main()
end= time.time()
print (end-start)

# Optional work

### Finding out the bounding boxes in the image

In [119]:
image = cv2.imread(r'E:\MS 4th Semester\Thesis\1st Meeting (8th Jan)\Syed Irtza Akhtar Bukhari - OCR patient monitor\irtza\frames58.jpg')
hsv_frame=hsv_conversion(image)
green=get_green_mask(hsv_frame,image)
red=get_red_mask(hsv_frame,image)
yellow=get_yellow_mask(hsv_frame,image)
white=get_white_mask(hsv_frame,image)

In [ ]:
res = reader.readtext(white,min_size= 80,text_threshold=0.90)

for (bbox, text, prob) in res: 
    (tl, tr, br, bl) = bbox
    tl = (int(tl[0]), int(tl[1]))
    tr = (int(tr[0]), int(tr[1]))
    br = (int(br[0]), int(br[1]))
    bl = (int(bl[0]), int(bl[1]))
    cv2.rectangle(white, tl, br, (0, 255, 0), 2)
#     cv2.putText(red, text, (tl[0], tl[1] - 10),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
plt.rcParams['figure.figsize'] = (16,16)
imS = cv2.resize(white, (1000, 600)) 
cv2.imshow('Bounding Rectangle',imS)
cv2.waitKey(0)